<a href="https://colab.research.google.com/github/benjaminbrown038/WeightAndBiases/blob/main/notebooks/wandb/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WANDB Experiments

#### Imports

In [1]:
%%capture
!pip3 install wandb -qU

In [2]:
import wandb
import random

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
total_runs = 5
for run in range(total_runs):
  wandb.init(project="basic-intro", name=f"experiment_{run}", config={"learning_rate": 0.02,"architecture": "CNN","dataset": "CIFAR-100","epochs": 10})

In [6]:
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset
      wandb.log({"acc": acc, "loss": loss})
  wandb.finish()

acc,▁▆▇█▇▇██
loss,█▅▁▃▂▁▁▂
acc,0.82123
loss,0.22939
